<a href="https://colab.research.google.com/github/dookda/cmu_udfire_gee/blob/main/predict_hp_using_lstm_gee_colab.ipynb_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api geemap folium tensorflow scikit-learn

import ee
import geemap
import folium
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten, TimeDistributed, Reshape
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
# เริ่มต้นใช้งาน GEE
ee.Authenticate()
try:
    ee.Initialize(project="ee-sakda-451407")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-sakda-451407")

In [3]:
# กำหนดพื้นที่ศึกษา (ตำบลสุเทพ)
roi = ee.Geometry.Rectangle([98.9, 18.7, 99.0, 18.8])  # ควรปรับค่าพิกัดให้ถูกต้อง

# กำหนดช่วงวันที่
start_date = '2020-01-01'
end_date = '2024-12-31'

# ดึงข้อมูล MODIS Hotspot (Thermal Anomalies)
modis = ee.ImageCollection('MODIS/061/MOD14A1') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# ดึงข้อมูลคุณลักษณะอื่นๆ ที่เกี่ยวข้อง
# เช่น อุณหภูมิพื้นผิว, ดัชนีพืชพรรณ ฯลฯ
modis_veg = ee.ImageCollection('MODIS/061/MOD13A2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# สร้างฟีเจอร์สำหรับการทำนาย
def add_date_info(image):
    date = ee.Date(image.get('system:time_start'))
    doy = date.getRelative('day', 'year')
    week = date.getRelative('week', 'year')
    return image.addBands(ee.Image.constant(doy).rename('doy').toInt()) \
               .addBands(ee.Image.constant(week).rename('week').toInt())

# เพิ่มข้อมูลวันและสัปดาห์
modis = modis.map(add_date_info)
modis_veg = modis_veg.map(add_date_info)

# รวมข้อมูลทั้งหมด
def combine_collections(img):
    # ดึงข้อมูล vegetation ที่ใกล้เคียงวันที่ที่สุด
    veg_img = modis_veg.filterDate(img.date(), img.date().advance(1, 'day')).first()

    # ตรวจสอบว่ามีข้อมูล vegetation หรือไม่
    veg_img = ee.Algorithms.If(
        ee.Image(veg_img).bandNames().size().gt(0),
        veg_img,
        ee.Image(0).rename('NDVI').addBands(ee.Image(0).rename('EVI'))
    )

    return img.addBands(ee.Image(veg_img).select(['NDVI', 'EVI']))

collection = modis.map(combine_collections)

# สร้างข้อมูลรายสัปดาห์
def create_weekly_composites(start_date, weeks):
    weekly_data = []
    for i in range(weeks):
        start = start_date.advance(i, 'week')
        end = start.advance(1, 'week')

        # ตรวจสอบว่ามีข้อมูลในช่วงเวลานี้หรือไม่
        weekly_coll = collection.filterDate(start, end)
        weekly_composite = ee.Algorithms.If(
            weekly_coll.size().gt(0),
            weekly_coll.mean(),
            ee.Image(0).rename('MaxFRP')
                .addBands(ee.Image(0).rename('FireMask'))
                .addBands(ee.Image(0).rename('NDVI'))
                .addBands(ee.Image(0).rename('EVI'))
                .addBands(ee.Image(i % 52).rename('week').toInt())
                .addBands(ee.Image(i % 365).rename('doy').toInt())
        )

        weekly_composite = ee.Image(weekly_composite).set('week_start', start.millis(), 'week_number', i)
        weekly_data.append(weekly_composite)

    return ee.ImageCollection(weekly_data)

start_date_ee = ee.Date(start_date)
total_weeks = 260  # 5 ปี * 52 สัปดาห์
weekly_collection = create_weekly_composites(start_date_ee, total_weeks)

In [4]:
# แปลงข้อมูลเป็น array
def image_to_array(image, region, scale):
    try:
        # ตรวจสอบว่า image เป็น valid
        image = ee.Algorithms.If(
            ee.Image(image).bandNames().size().gt(0),
            image,
            ee.Image(0).rename('MaxFRP')
                .addBands(ee.Image(0).rename('FireMask'))
                .addBands(ee.Image(0).rename('NDVI'))
                .addBands(ee.Image(0).rename('EVI'))
                .addBands(ee.Image(0).rename('week').toInt())
                .addBands(ee.Image(0).rename('doy').toInt())
        )

        arr = ee.Image(image).sampleRectangle(region=region, defaultValue=0)
        return arr
    except Exception as e:
        print(f"Error: {e}")
        return None

# ดึงข้อมูลเป็น numpy array
scale = 1000  # 1km resolution
bands = ['MaxFRP', 'FireMask', 'NDVI', 'EVI', 'doy', 'week']

# สร้างข้อมูล training
training_data = []
labels = []

for week in range(total_weeks - 1):  # ลบ 1 เพราะเราจะใช้ทำนายสัปดาห์ถัดไป
    current_week_img = weekly_collection.filter(ee.Filter.eq('week_number', week)).first()
    next_week_img = weekly_collection.filter(ee.Filter.eq('week_number', week + 1)).first()

    # ดึงข้อมูล hotspot ของสัปดาห์ถัดไป (เป็น label)
    hotspot_next = ee.Image(next_week_img).select('MaxFRP').gt(0)  # 1 มี hotspot, 0 ไม่มี

    # ดึงข้อมูล features ของสัปดาห์ปัจจุบัน
    features = ee.Image(current_week_img).select(bands)

    # แปลงเป็น array
    features_arr = image_to_array(features, roi, scale)
    hotspot_arr = image_to_array(hotspot_next, roi, scale)

    if features_arr and hotspot_arr:
        # เนื่องจากพื้นที่อาจมีขนาดเล็ก เราอาจต้องปรับขนาดข้อมูล
        training_data.append(features_arr.getInfo())
        labels.append(hotspot_arr.getInfo())

# แปลงเป็น numpy array
X = np.array(training_data)
y = np.array(labels)

# ปรับขนาดข้อมูลให้เหมาะสมกับแบบจำลอง
def create_sequences(data, labels, time_steps=4):
    X_seq, y_seq = [], []
    for i in range(time_steps, len(data)):
        X_seq.append(data[i-time_steps:i])
        y_seq.append(labels[i])
    return np.array(X_seq), np.array(y_seq)

time_steps = 4  # ใช้ข้อมูล 4 สัปดาห์ย้อนหลังเพื่อทำนายสัปดาห์ถัดไป
X_seq, y_seq = create_sequences(X, y, time_steps)

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
split_ratio = 0.8
split_idx = int(len(X_seq) * split_ratio)

X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

EEException: reduce.mean: Error in map(ID=2022_02_09):
Image.select: Parameter 'input' is required and may not be null.

In [ ]:
# กำหนดขนาด input
n_timesteps, n_rows, n_cols, n_channels = X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4]

# สร้างแบบจำลอง CNN-LSTM
model = Sequential()

# CNN layers สำหรับสกัดคุณลักษณะเชิงพื้นที่
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'),
                         input_shape=(n_timesteps, n_rows, n_cols, n_channels)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

# LSTM layers สำหรับเรียนรู้รูปแบบเชิงเวลา
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))

# Fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # binary classification

# compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# แสดงโครงสร้างแบบจำลอง
model.summary()

In [ ]:
# ฝึกแบบจำลอง
history = model.fit(X_train, y_train,
                   epochs=50,
                   batch_size=16,
                   validation_data=(X_test, y_test),
                   verbose=1)

# ประเมินแบบจำลอง
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# คำนวณ metrics
accuracy = accuracy_score(y_test.flatten(), y_pred.flatten())
precision = precision_score(y_test.flatten(), y_pred.flatten())
recall = recall_score(y_test.flatten(), y_pred.flatten())
f1 = f1_score(y_test.flatten(), y_pred.flatten())
conf_matrix = confusion_matrix(y_test.flatten(), y_pred.flatten())

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

# พล็อตการเรียนรู้
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# ดึงข้อมูลล่าสุดสำหรับการทำนาย
latest_data = X[-time_steps:]  # ใช้ข้อมูล time_steps สัปดาห์ล่าสุด
latest_data = latest_data.reshape(1, time_steps, n_rows, n_cols, n_channels)

# ทำนาย
prediction = model.predict(latest_data)
hotspot_prediction = (prediction > 0.5).astype(int)

# สร้าง raster จากผลการทำนาย
def array_to_raster(array, transform, crs='EPSG:4326'):
    """แปลง numpy array เป็น raster"""
    from rasterio.transform import from_origin
    import rasterio

    # สร้าง transform จากข้อมูลพื้นที่
    # ควรปรับค่าเหล่านี้ตามพื้นที่จริง
    transform = from_origin(roi.getInfo()['coordinates'][0][0][0],
                           roi.getInfo()['coordinates'][0][0][1],
                           scale, scale)

    with rasterio.open('prediction.tif', 'w',
                       driver='GTiff',
                       height=array.shape[0],
                       width=array.shape[1],
                       count=1,
                       dtype=rasterio.uint8,
                       crs=crs,
                       transform=transform) as dst:
        dst.write(array.astype(rasterio.uint8), 1)

# บันทึกผลการทำนายเป็น raster
array_to_raster(hotspot_prediction[0, :, :, 0], None)

In [ ]:
# สร้างแผนที่ Folium
m = folium.Map(location=[18.78, 98.95], zoom_start=12)  # ควรปรับตำแหน่งให้ตรงกับพื้นที่ศึกษา

# เพิ่มผลการทำนายลงบนแผนที่
prediction_layer = folium.raster_layers.TileLayer(
    tiles='prediction.tif',
    name='Hotspot Prediction',
    opacity=0.7,
    attr='Hotspot Prediction'
)
prediction_layer.add_to(m)

# เพิ่ม layer control
folium.LayerControl().add_to(m)

# บันทึกแผนที่
m.save('hotspot_prediction_map.html')

# แสดงแผนที่
m

In [ ]:
# บันทึกแบบจำลอง
model.save('hotspot_cnn_lstm_model.h5')

# โหลดแบบจำลองที่ฝึกไว้
from tensorflow.keras.models import load_model
loaded_model = load_model('hotspot_cnn_lstm_model.h5')